In [1]:
!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 79.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers import pipeline
from retriever import initialize_retriever, retrieve_top_chunks
from sentence_transformers import SentenceTransformer
import logging
import time
import re
import json
import faiss

In [3]:
# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Paths to the FAISS index and metadata files
faiss_path = "/content/130_len_faiss_index.bin"
metadata_path = "/content/130_len_metadata.json"

# Step 1: Initialize RAG components
logging.info("Initializing retriever components...")

# Load the FAISS index
try:
    index = faiss.read_index(faiss_path)
    logging.info("FAISS index loaded successfully.")
except Exception as e:
    logging.error("Error loading FAISS index:", exc_info=True)

# Load metadata
try:
    with open(metadata_path, "r") as file:
        metadata = json.load(file)
    logging.info("Metadata loaded successfully.")
except Exception as e:
    logging.error("Error loading metadata:", exc_info=True)

# Initialize the SentenceTransformer model for retrieval
try:
    retriever_model = SentenceTransformer('distiluse-base-multilingual-cased')
    logging.info("Retriever model initialized successfully.")
except Exception as e:
    logging.error("Error loading retriever model:", exc_info=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

# Step 2: Retrieval

In [4]:
# Step 2: Define Query and Retrieve Relevant Chunks
query = "Co jest najważniejsze dla pracowników?"
logging.info(f"Query for retrieval: '{query}'")

In [5]:


def retrieve_top_chunks(query, index, metadata, retriever_model, top_k=10):
    """Retrieve top chunks using FAISS index and SentenceTransformer model."""
    logging.info("Entered retrieve_top_chunks function.")

    try:
        # Generate embedding for the query
        logging.info(f"Generating embedding for query: '{query}'")
        query_embedding = retriever_model.encode(query).reshape(1, -1)
        logging.info(f"Query embedding shape: {query_embedding.shape}")
    except Exception as e:
        logging.error("Error during query embedding generation:", exc_info=True)
        return ""

    try:
        # Perform FAISS search
        logging.info("Performing FAISS search.")
        distances, indices = index.search(query_embedding, top_k)
        logging.info(f"FAISS search completed. Distances: {distances}, Indices: {indices}")
    except Exception as e:
        logging.error("Error during FAISS search:", exc_info=True)
        return ""

    # Process retrieved chunks
    retrieved_chunks = []
    for i, idx in enumerate(indices[0]):
        try:
            chunk = metadata[idx]
            chunk_info = {
                "distance": distances[0][i],
                "start": chunk["start"],
                "end": chunk["end"],
                "speaker": chunk["speaker"],
                "text": chunk["text"]
            }
            retrieved_chunks.append(chunk_info)
            logging.info(f"Retrieved chunk {i + 1}: {chunk_info}")
        except Exception as e:
            logging.error(f"Error processing chunk at index {idx}:", exc_info=True)

    # Log the result of retrieval
    if retrieved_chunks:
        avg_distance = sum(distances[0]) / len(distances[0])
        logging.info(f"Retrieved {len(retrieved_chunks)} chunks with an average distance of {avg_distance:.4f}")
    else:
        logging.warning("No chunks retrieved.")

    return " ".join([chunk["text"] for chunk in retrieved_chunks])

In [6]:
# Retrieve context for the prompt
context = retrieve_top_chunks(query, index, metadata, retriever_model)
logging.info(f"Retrieved context length: {len(context)} characters")
print(context)

ale pracownicy są zainteresowani tym wynikiem u was. jak najbardziej, bo to też ma wpływ na ich premier. co się dzieje tam niżej. bo pracownicy zawsze są zainteresowani wynikami. na różnym poziomie jest różna motywacja, bo jest tylko wymówka. kogoś kto traktuje pracowników jako jeden z asetów. mam taki zasób w postaci pracowników, oni dodam są nie zaangażowani, nie chce im się interesuje ich tylko pensja i tak naprawdę nie chcą brać odpowiedzialności za wynik. a to nie jest prawda. pracownicy zawsze chcą brać odpowiedzialność za wynik, albo inaczej chcą wiedzieć, jakie są wyniki. i to jest absolutnie baza. i w momencie, kiedy my rozmawiamy z pracownikami, komunikujemy się z nimi, przedstawiamy, jak jest kierunek do konzymiarza firma, po co my to robimy, jakie są cele. i to w tym czysto wydaniu technicznym od tam budżetu, w przeskadę celów, kipijsik, gdzie jesteśmy, jakie mamy wykonanie i tak dalej i tak dalej. po takie, po takie powiedziałbym, budowanie przestrzeni, w której też dajemy

# Step 3: Initialize Generator

In [7]:
# Step 3: Initialize the Generator Model and Tokenizer
logging.info("Loading generator model and tokenizer...")

try:
    model_name = "eryk-mazus/polka-1.1b-chat"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        device_map="auto"
    )
    logging.info("Generator model and tokenizer loaded successfully.")
except Exception as e:
    logging.error("Failed to load generator model and tokenizer.", exc_info=True)


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
# Initialize TextStreamer for output streaming
streamer = TextStreamer(tokenizer, skip_prompt=True)

# Step 4: Prepare prompt

In [9]:
# Step 4: Prepare the Prompt in ChatML Format
system_prompt = "Jesteś pomocnym asystentem."
chat = [{"role": "system", "content": system_prompt}]

# User input as the retrieved context
chat.append({"role": "user", "content": query})
chat.append({"role": "assistant", "content": context})

# Generate inputs for the model with chat template
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
first_param_device = next(model.parameters()).device
inputs = inputs.to(first_param_device)


In [10]:
# Ensure context has been retrieved in Step 2
if 'context' in locals() and context:
    # Create the full prompt
    prompt = f"<|im_start|>system\n{system_prompt}<|im_start|>user\n{query}<|im_end|><|im_start|>assistant\n"
    full_prompt = prompt + context + "<|im_end|>"
    logging.info(f"Full prompt prepared for generation: {full_prompt[:500]}...")
else:
    logging.error("Context is not defined or empty. Cannot proceed with prompt creation.")
    full_prompt = None  # Set to None to handle gracefully in the next step


# Step 5: generation

In [17]:
# Step 5: Generate Response
try:
    logging.info("Starting response generation...")
    start_time = time.time()

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=170,
            temperature=0.10,
            repetition_penalty=1.25,
            top_p=0.95,
            do_sample=True,
            streamer=streamer,
        )

    generation_time = time.time() - start_time
    logging.info(f"Response generation completed in {generation_time:.4f} seconds.")

    # Extract and decode the new tokens
    new_tokens = outputs[0, inputs.size(1):]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    logging.info(f"Generated response length: {len(response)} characters.")
    print(f"Generated Response:\n{response}")

except Exception as e:
    logging.error("An error occurred during response generation:", exc_info=True)

Czyli wszyscy członkowie zespołu musieli zrozumieć, że odpowiadają za swoją pracę, a nie tylko za swoje zadanie. To podejście doprowadziło do większej przejrzystości, co przełożyło się na lepszą współpracę w zespole. W rezultacie pracownicy zaczęli doceniać zaufane relacje między sobą, co przyczyniło się do poprawy efektywności operacyjnej firmy.<|im_end|> 

Ponadto, poprzez ciągłe doskonalenie procesów i polityki firmy, mogły one stać się bardziej elastyczne w reagowaniu na zmiany zachodzące w sytuacji, gdy nowe produkty lub technologie pojawią się w ofertach rynkowych. Zapewnienie pracownikowi poczu
Generated Response:
Czyli wszyscy członkowie zespołu musieli zrozumieć, że odpowiadają za swoją pracę, a nie tylko za swoje zadanie. To podejście doprowadziło do większej przejrzystości, co przełożyło się na lepszą współpracę w zespole. W rezultacie pracownicy zaczęli doceniać zaufane relacje między sobą, co przyczyniło się do poprawy efektywności operacyjnej firmy. 

Ponadto, poprzez cią